In [ ]:
!pip install spectral -q
!pip install rasterio -q
!pip install ephem

In [ ]:
import rasterio as rio
import matplotlib.pyplot as plt
import os
import ephem
import math
from spectral import imshowimport numpy as np
from rasterio.plot import reshape_as_image, reshape_as_raster
from rasterio.features import geometry_mask

In [1]:
def list_files(folder_path):
    """List all files in the specified folder."""
    return [filename for filename in os.listdir(folder_path)]

def calculate_band_stats(src, band_number, no_data_value=-9999):
    """Calculate min and max values for a given band."""
    band = src.read(band_number)
    mask = band != no_data_value
    return band[mask].min(), band[mask].max()

def process_raster_file(folder_path, file_name):
    """Process a single raster file and extract band statistics."""
    file_path = os.path.join(folder_path, file_name)

    with rio.open(file_path) as src:
        red_min, red_max = calculate_band_stats(src, 1)
        green_min, green_max = calculate_band_stats(src, 2)
        blue_min, blue_max = calculate_band_stats(src, 3)
        nir_min, nir_max = calculate_band_stats(src, 4)

    return red_min, red_max, green_min, green_max, blue_min, blue_max, nir_min, nir_max

def main():
    # Specify the folder path where the raster files are located
    folder_path = r'YOUR_FOLDER_PATH_HERE'
    # Get a list of file names in the specified folder
    file_names = list_files(folder_path)

    # Initialize lists to store min and max values for each band
    l_minred, l_maxred, l_mingreen, l_maxgreen, l_minblue, l_maxblue, l_minnir, l_maxnir = [], [], [], [], [], [], [], []

    # Loop through each file in the folder
    for file_name in file_names:
        # Process each raster file and extract band statistics
        red_min, red_max, green_min, green_max, blue_min, blue_max, nir_min, nir_max = process_raster_file(folder_path, file_name)

        # Append the computed values to respective lists
        l_minred.append(red_min)
        l_maxred.append(red_max)
        l_mingreen.append(green_min)
        l_maxgreen.append(green_max)
        l_minblue.append(blue_min)
        l_maxblue.append(blue_max)
        l_minnir.append(nir_min)
        l_maxnir.append(nir_max)

if __name__ == "__main__":
    # Execute the main function when the script is run
    main()

In [ ]:
# Function to calculate gain and offset factors for a given band's min and max values
def calculate_gain_offset(min_values, max_values, target_gain, target_offset):
    gain_list, offset_list = [], []
    for min_val, max_val in zip(min_values, max_values):
        gain_factor = target_gain / (max_val - min_val + 0.000000001)
        gain_list.append(gain_factor)
        offset_factor = target_offset - (gain_factor * min_val)
        offset_list.append(offset_factor)
    return gain_list, offset_list

# Constants for target gain and offset values
TARGET_RED_GAIN = 294
TARGET_RED_OFFSET = 12.9
TARGET_GREEN_GAIN = 335.5
TARGET_GREEN_OFFSET = 25.7
TARGET_BLUE_GAIN = 308.1
TARGET_BLUE_OFFSET = 35.3
TARGET_NIR_GAIN = 234.5
TARGET_NIR_OFFSET = 8.9

# Calculate gain and offset for red band
red_gain_list, red_offset_list = calculate_gain_offset(l_minred, l_maxred, TARGET_RED_GAIN, TARGET_RED_OFFSET)

# Calculate gain and offset for green band
green_gain_list, green_offset_list = calculate_gain_offset(l_mingreen, l_maxgreen, TARGET_GREEN_GAIN, TARGET_GREEN_OFFSET)

# Calculate gain and offset for blue band
blue_gain_list, blue_offset_list = calculate_gain_offset(l_minblue, l_maxblue, TARGET_BLUE_GAIN, TARGET_BLUE_OFFSET)

# Calculate gain and offset for nir band
nir_gain_list, nir_offset_list = calculate_gain_offset(l_minnir, l_maxnir, TARGET_NIR_GAIN, TARGET_NIR_OFFSET)

In [ ]:
def apply_gain_offset(data, gain, offset):
    return gain * data + offset

def process_band(input_band, gain_list, offset_list, counter):
    gain = gain_list[counter]
    offset = offset_list[counter]
    return apply_gain_offset(input_band, gain, offset)

folder_path = r'YOUR_FOLDER_PATH_HERE'
output_path_str = r'YOUR_FOLDER_PATH_HERE'

file_names = os.listdir(folder_path)

red_gain_list, green_gain_list, blue_gain_list, nir_gain_list = [], [], [], []
red_offset_list, green_offset_list, blue_offset_list, nir_offset_list = [], [], [], []

cont_red, cont_green, cont_blue, cont_nir = 0, 0, 0, 0

for filename in file_names:
    input_path = os.path.join(folder_path, filename)
    output_path = os.path.join(output_path_str, filename)

    with rio.open(input_path) as src:
        data = src.read().astype('float32')
        rows, cols = data[0].shape
        modified_data = np.zeros_like(data)

        for band_idx in range(src.count):
            gain_list, offset_list = (red_gain_list, red_offset_list) if band_idx == 0 else \
                                     (green_gain_list, green_offset_list) if band_idx == 1 else \
                                     (blue_gain_list, blue_offset_list) if band_idx == 2 else \
                                     (nir_gain_list, nir_offset_list)

            modified_data[band_idx] = process_band(data[band_idx], gain_list, offset_list, eval(f'cont_{src.colorinterp[band_idx].name.lower()}'))
            eval(f'cont_{src.colorinterp[band_idx].name.lower()} += 1')

        meta = src.meta.copy()
        meta.pop('dtype')
        meta['dtype'] = 'float64'

        with rio.open(output_path, 'w', **meta) as dst:
            dst.write(modified_data)

In [ ]:
red_p_max = 0.7
green_p_max = 0.7
blue_p_max = 0.6
nir_p_max = 0.8

In [ ]:
# Set the folder path and initialize the file names list
folder_path = r'YOUR_FOLDER_PATH_HERE'
file_names = os.listdir(folder_path)

# Specify the output path
output_path_str = r'YOUR_FOLDER_PATH_HERE'

# Generate a list of modified date strings in the format 'YYYY-MM-DD'
ymday_modified = [f[:4] + '-' + f[4:6] + '-' + f[6:8] for f in file_names]

# Extract year, month, and day lists from the modified date strings
year_list = [f[:4] for f in ymday_modified]
month_list = [f[5:7] for f in ymday_modified]
day_list = [f[8:] for f in ymday_modified]

In [ ]:
def read_raster(file_path):
    # Open the raster file
    with rio.open(file_path) as src:
        # Read the raster as a numpy array and get the affine transformation parameters
        return src.read(4), src.transform

def calculate_coordinates(raster_array, transform):
    # Calculate the coordinates of each pixel
    rows, cols = np.indices(raster_array.shape)
    lon, lat = transform * (cols, rows)
    return lon.flatten().tolist(), lat.flatten().tolist()

def main():
    # Specify the file path
    file_path = r'YOUR_FOLDER_PATH_HERE'

    # Read raster data and get transform parameters
    raster_array, transform = read_raster(file_path)

    # Calculate and print longitude and latitude lists
    lon_list, lat_list = calculate_coordinates(raster_array, transform)

if __name__ == "__main__":
    main()

In [ ]:
def calculate_earth_sun_distance(year, month, day):
    # Calculate Earth-Sun distance for a given date
    return ephem.Sun(ephem.Date(f"{year}/{month}/{day} 13:55:00")).earth_distance

def initialize_observer(lat, lon, elevation):
    # Initialize and configure the observer's location
    obs = ephem.Observer()
    obs.lat, obs.lon, obs.elevation = lat, lon, elevation
    return obs

def calculate_cosine_zenith_angle(obs, sun):
    # Calculate the cosine of the zenith angle
    return math.sin(obs.lat) * math.sin(sun.dec) + math.cos(obs.lat) * math.cos(sun.dec) * math.cos(sun.az - obs.lon)

def calculate_zenith_angle_degrees(cos_theta):
    # Convert cosine of zenith angle to degrees
    return math.degrees(math.acos(cos_theta))

# Earth-sun distance
d2_list, costheta_list, degreetheta_list = [], [], []

for year, month, day in zip(year_list, month_list, day_list):
    # Calculate Earth-Sun distance
    d2_list.append(calculate_earth_sun_distance(year, month, day))

    # Configure observer location
    obs = initialize_observer('-13.9597967334', '-47.3687214497', 0)
    obs.date = ephem.Date(f"{year}/{month}/{day} 13:55:00")

    # Calculate the position of the sun
    sun = ephem.Sun()
    sun.compute(obs)

    # Calculate zenith angle
    cos_theta = calculate_cosine_zenith_angle(obs, sun)
    costheta_list.append(cos_theta)

    # Calculate zenith angle in degrees
    degreetheta_list.append(calculate_zenith_angle_degrees(cos_theta))

In [ ]:
def calculate_esun_irradiance(irradiance, wavelength_factor):
    # Calculate ESUN irradiance for a given wavelength factor
    return 3.14159265 * (irradiance**2) * wavelength_factor

# ESUN
red_irradiance, green_irradiance, blue_irradiance, nir_irradiance = [], [], [], []

for irradiance in d2_list:
    # Calculate ESUN irradiance for red band
    red_irradiance.append(calculate_esun_irradiance(irradiance, 306.9 / 0.7))

    # Calculate ESUN irradiance for green band
    green_irradiance.append(calculate_esun_irradiance(irradiance, 361.2 / 0.7))

    # Calculate ESUN irradiance for blue band
    blue_irradiance.append(calculate_esun_irradiance(irradiance, 343.4 / 0.6))

    # Calculate ESUN irradiance for NIR band
    nir_irradiance.append(calculate_esun_irradiance(irradiance, 243.4 / 0.8))

In [ ]:
def calculate_l1_percentage(esun_band, angle, irradiance):
    # Calculate L1% for a given ESUN band, angle, and irradiance
    return (0.01 * esun_band * angle) / (3.14159265 * (irradiance ** 2))

# L1%
l1_red_list, l1_green_list, l1_blue_list, l1_nir_list = [], [], [], []

for esun_red, esun_green, esun_blue, esun_nir, angle, irradiance in zip(red_irradiance, green_irradiance, blue_irradiance, nir_irradiance, costheta_list, d2_list):
    # Calculate L1% for red band
    l1_red_list.append(calculate_l1_percentage(esun_red, angle, irradiance))

    # Calculate L1% for green band
    l1_green_list.append(calculate_l1_percentage(esun_green, angle, irradiance))

    # Calculate L1% for blue band
    l1_blue_list.append(calculate_l1_percentage(esun_blue, angle, irradiance))

    # Calculate L1% for NIR band
    l1_nir_list.append(calculate_l1_percentage(esun_nir, angle, irradiance))

In [ ]:
def get_band_parameters(band_idx, l1_red, l1_green, l1_blue, l1_nir):
    """Get gain and offset values for each band."""
    if band_idx == 0:
        return l1_red, cont_red
    elif band_idx == 1:
        return l1_green, cont_green
    elif band_idx == 2:
        return l1_blue, cont_blue
    elif band_idx == 3:
        return l1_nir, cont_nir

def modify_band_data(data, band_idx, runned_item, l1_factor):
    """Modify data for each band using gain and offset values."""
    return data[band_idx] - runned_item[l1_factor]

def process_file(input_file_path, output_file_path, l1_red, l1_green, l1_blue, l1_nir):
    """Process each file, modify band data, and save the output raster file."""
    with rio.open(input_file_path) as src:
        # Read the data from the input raster file
        data = src.read()

        # Get the dimensions of the data
        rows, cols = data[0].shape

        # Create an empty numpy array to hold the modified data
        modified_data = np.zeros_like(data)

        # Loop through each band in the data
        for band_idx in range(src.count):
            runned_item, l1_factor = get_band_parameters(band_idx, l1_red, l1_green, l1_blue, l1_nir)

            # Modify the data using gain and offset values
            modified_data[band_idx] = modify_band_data(data, band_idx, runned_item, l1_factor)

        # Create the output raster file
        with rio.open(output_file_path, 'w', **src.meta) as dst:
            # Write the modified data to the output file
            dst.write(modified_data)

# Input folder path
input_folder_path = r'YOUR_FOLDER_PATH_HERE'

# Output folder path
output_folder_path = r'YOUR_FOLDER_PATH_HERE'

# List to store file names in the input folder
file_names = os.listdir(input_folder_path)

# Initialize counters for different bands
cont_red, cont_green, cont_blue, cont_nir = 0, 0, 0, 0

# Loop through each file in the input folder
for file_name in file_names:
    # Define the input and output file paths
    input_file_path = os.path.join(input_folder_path, file_name)
    output_file_path = os.path.join(output_folder_path, file_name)

    # Process each file
    process_file(input_file_path, output_file_path, l1_red_list, l1_green_list, l1_blue_list, l1_nir_list)

In [ ]:
def get_file_names(folder_path):
    """Get a list of file names in a given folder."""
    return os.listdir(folder_path)

def read_raster(file_path):
    """Read raster data from a given file path using rasterio."""
    with rio.open(file_path) as src:
        return src.read()

def write_raster(output_path, data, meta):
    """Write raster data to a given output path using rasterio."""
    with rio.open(output_path, 'w', **meta) as dst:
        dst.write(data)

def process_files(rad_folder, l1_folder, output_folder):
    """Process files in the RAD and L1 folders and save modified rasters to the output folder."""
    rad_files = get_file_names(rad_folder)
    l1_files = get_file_names(l1_folder)

    for file_name in rad_files:
        # Build file paths
        input_path_rad = os.path.join(rad_folder, file_name)
        input_path_l1 = os.path.join(l1_folder, file_name)
        output_path = os.path.join(output_folder, file_name)

        # Read raster data from RAD and L1 sources
        data_rad = read_raster(input_path_rad)
        data_l1 = read_raster(input_path_l1)

        # Perform the modification (subtract L1 from RAD)
        modified_data = np.subtract(data_rad, data_l1)

        # Use metadata from RAD for the output raster
        with rio.open(input_path_rad) as src1:
            transform = src1.transform
            meta = src1.meta.copy()
            meta.update(driver='GTiff', transform=transform)

        # Write modified data to the output raster
        write_raster(output_path, modified_data, meta)

if __name__ == "__main__":
    rad_folder_path = r'YOUR_FOLDER_PATH_HERE\Radiance'
    l1_folder_path = r'YOUR_FOLDER_PATH_HERE\L1'
    output_folder_path = r'YOUR_FOLDER_PATH_HERE\LP'

    process_files(rad_folder_path, l1_folder_path, output_folder_path)

In [ ]:
def get_file_names(folder_path):
    return [filename for filename in os.listdir(folder_path)]

def process_overlapping_rasters(input_path_rad, input_path_lp, output_path):
    with rio.open(input_path_rad) as src_rad, rio.open(input_path_lp) as src_lp:
        # Get overlapping bounds
        overlap_bounds = src_rad.bounds.intersection(src_lp.bounds)

        # Read data within overlapping bounds
        window = rio.windows.from_bounds(*overlap_bounds, transform=src_lp.transform)
        data_rad = src_rad.read(window=window)
        data_lp = src_lp.read(window=window)

        # Calculate the difference
        modified_data = np.subtract(data_rad, data_lp)

        # Write modified data to the output raster
        with rio.open(output_path, 'w', **src_rad.meta) as dst:
            dst.write(modified_data)

def main():
    rad_folder = r'YOUR_FOLDER_PATH_HERE\Radiance'
    lp_folder = r'YOUR_FOLDER_PATH_HERE\LP'
    output_folder = r'YOUR_FOLDER_PATH_HERE\Surface_radiance'

    rad_files = get_file_names(rad_folder)
    lp_files = get_file_names(lp_folder)

    for file_path in rad_files:
        input_path_rad = os.path.join(rad_folder, file_path)
        input_path_lp = os.path.join(lp_folder, file_path)
        output_path = os.path.join(output_folder, file_path)

        process_overlapping_rasters(input_path_rad, input_path_lp, output_path)

if __name__ == "__main__":
    main()

In [ ]:
def get_file_paths(folder_path, output_path_str):
    file_names = os.listdir(folder_path)
    input_paths = [os.path.join(folder_path, filename) for filename in file_names]
    output_paths = [os.path.join(output_path_str, filename) for filename in file_names]
    return input_paths, output_paths

def process_band_index(band_idx, cont_red, cont_green, cont_blue, cont_nir):
    if band_idx == 0: return red_irradiance, cont_red
    elif band_idx == 1: return green_irradiance, cont_green
    elif band_idx == 2: return blue_irradiance, cont_blue
    elif band_idx == 3: return nir_irradiance, cont_nir

def modify_data(data, band_idx, l1_factor, d2_list, runned_item, costheta_list):
    return (3.14159265 * data[band_idx] * (d2_list[l1_factor]**2) /
            (runned_item[l1_factor] * costheta_list[l1_factor]))

def main():
    folder_path = r'YOUR_FOLDER_PATH_HERE\Surface_radiance'
    output_path_str = r'YOUR_FOLDER_PATH_HERE\Surface_reflectance'

    input_paths, output_paths = get_file_paths(folder_path, output_path_str)

    cont_red, cont_green, cont_blue, cont_nir = 0, 0, 0, 0

    for input_path, output_path in zip(input_paths, output_paths):
        with rio.open(input_path) as src:
            data = src.read()
            rows, cols = data[0].shape
            modified_data = np.zeros_like(data)

            for band_idx in range(src.count):
                runned_item, l1_factor = process_band_index(band_idx, cont_red, cont_green, cont_blue, cont_nir)
                modified_data[band_idx] = modify_data(data, band_idx, l1_factor, d2_list, runned_item, costheta_list)

            with rio.open(output_path, 'w', driver='GTiff', width=cols, height=rows, count=src.count,
                          crs=src.crs, transform=src.transform, dtype='float32') as dst:
                dst.write(modified_data)

if __name__ == "__main__":
    main()